In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Load CSV data from Part I into a dataframe
weather_df = pd.read_csv('output_data/cities.csv').drop(['Unnamed: 0'],axis=1)
weather_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,carnarvon,AU,-24.8667,113.6333,69.80,52,0,8.19
1,ushuaia,AR,-54.8000,-68.3000,42.80,87,75,8.05
2,georgetown,MY,5.4112,100.3354,76.24,94,40,4.61
3,dikson,RU,73.5069,80.5464,31.80,98,100,17.60
4,punta arenas,CL,-53.1500,-70.9167,44.29,93,75,2.30
...,...,...,...,...,...,...,...,...
579,sechura,PE,-5.5569,-80.8222,72.43,69,41,15.59
580,dukat,RU,62.5500,155.5500,21.67,67,2,4.76
581,osinovo,RU,55.8809,48.8810,33.80,64,0,4.47
582,bloomington,US,44.8408,-93.2983,47.03,81,90,13.80


In [2]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Pull data from main dataframe for heatmap layer
locations = weather_df[['Latitude','Longitude']].astype(float)
humidity_weight = weather_df['Humidity (%)']

figure_layout = {'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'}

fig=gmaps.figure(figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight,
                                 dissipating=False, max_intensity=100,
                                point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
# Create Dataframe fitting weather criteria
best_vacation = weather_df.loc[(weather_df['Temperature (F)'] <= 80) & (weather_df['Temperature (F)'] >= 70) & 
                               (weather_df['Wind Speed (mph)'] <= 10) & (weather_df['Cloudiness (%)'] == 0), :]

# Drop any rows with missing data
best_vacation.dropna(how='any')

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
66,beloha,MG,-25.1667,45.0500,70.27,74,0,5.50
284,inhambane,MZ,-23.8650,35.3833,71.20,88,0,4.85
339,gulariya,NP,28.2333,81.3333,75.00,75,0,1.01
370,mahmudabad,IN,27.3000,81.1167,71.60,56,0,4.70
416,porbandar,IN,21.6422,69.6093,78.80,94,0,5.75
535,namibe,AO,-15.1961,12.1522,74.46,73,0,7.11


In [4]:
# Hotel Map
hotel_df = weather_df.copy()
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,carnarvon,AU,-24.8667,113.6333,69.80,52,0,8.19
1,ushuaia,AR,-54.8000,-68.3000,42.80,87,75,8.05
2,georgetown,MY,5.4112,100.3354,76.24,94,40,4.61
3,dikson,RU,73.5069,80.5464,31.80,98,100,17.60
4,punta arenas,CL,-53.1500,-70.9167,44.29,93,75,2.30
...,...,...,...,...,...,...,...,...
579,sechura,PE,-5.5569,-80.8222,72.43,69,41,15.59
580,dukat,RU,62.5500,155.5500,21.67,67,2,4.76
581,osinovo,RU,55.8809,48.8810,33.80,64,0,4.47
582,bloomington,US,44.8408,-93.2983,47.03,81,90,13.80


In [5]:
# geocoordinates
hotel_name = []
hotel_address = []
lats = hotel_df['Latitude']
lngs = hotel_df['Longitude']
x=0

for city in hotel_df['City']:
    target_coordinates = f'{lats[x]},{lngs[x]}'
    target_radius = 5000
    target_type = "lodging"
    x += 1

    # Set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)

    temp = response.json()
    try:
        hotel_name.append(temp['results'][0]['name'])
        hotel_address.append(temp['results'][0]['vicinity'])
        print(f'Hotel in {city} found! Finding address...')
    except:
        hotel_name.append('N/A')
        hotel_address.append('N/A')
        print(f'No hotels in {city}.')
        
hotel_df['Hotel Name'] = hotel_name
hotel_df['Hotel Address'] = hotel_address

Hotel in carnarvon found! Finding address...
Hotel in ushuaia found! Finding address...
Hotel in georgetown found! Finding address...
No hotels in dikson.
Hotel in punta arenas found! Finding address...
Hotel in airai found! Finding address...
Hotel in barrow found! Finding address...
No hotels in jurm.
Hotel in hasaki found! Finding address...
Hotel in lagoa found! Finding address...
Hotel in havelock found! Finding address...
Hotel in rikitea found! Finding address...
Hotel in puerto ayora found! Finding address...
Hotel in ambilobe found! Finding address...
Hotel in mataura found! Finding address...
Hotel in coquimbo found! Finding address...
Hotel in ribeira grande found! Finding address...
Hotel in klaksvik found! Finding address...
Hotel in bayshore gardens found! Finding address...
Hotel in maceio found! Finding address...
Hotel in hilo found! Finding address...
No hotels in albany.
Hotel in upernavik found! Finding address...
Hotel in atuona found! Finding address...
Hotel in l

Hotel in veraval found! Finding address...
Hotel in margate found! Finding address...
Hotel in grand gaube found! Finding address...
Hotel in vila franca do campo found! Finding address...
Hotel in freeport found! Finding address...
Hotel in kochi found! Finding address...
Hotel in dakoro found! Finding address...
Hotel in geraldton found! Finding address...
No hotels in saskylakh.
No hotels in zima.
Hotel in roma found! Finding address...
Hotel in krasnoselkup found! Finding address...
Hotel in wencheng found! Finding address...
Hotel in faya found! Finding address...
No hotels in chokurdakh.
Hotel in hithadhoo found! Finding address...
Hotel in rocha found! Finding address...
Hotel in terrace bay found! Finding address...
Hotel in hovd found! Finding address...
Hotel in nanortalik found! Finding address...
Hotel in cabo san lucas found! Finding address...
Hotel in bridlicna found! Finding address...
Hotel in athabasca found! Finding address...
Hotel in twentynine palms found! Finding

Hotel in victoria found! Finding address...
Hotel in nyurba found! Finding address...
Hotel in tuatapere found! Finding address...
Hotel in sabha found! Finding address...
Hotel in codrington found! Finding address...
Hotel in sovetskiy found! Finding address...
Hotel in korla found! Finding address...
Hotel in hearst found! Finding address...
Hotel in calatayud found! Finding address...
Hotel in villa bruzual found! Finding address...
Hotel in tres lagoas found! Finding address...
Hotel in vila found! Finding address...
Hotel in tarauaca found! Finding address...
Hotel in constitucion found! Finding address...
Hotel in sao joao da barra found! Finding address...
Hotel in zhezkazgan found! Finding address...
Hotel in leon found! Finding address...
Hotel in kavieng found! Finding address...
Hotel in westport found! Finding address...
Hotel in havre-saint-pierre found! Finding address...
Hotel in nizhniy odes found! Finding address...
Hotel in durban found! Finding address...
Hotel in da

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))